# Knet RNN example

In [31]:
# After installing and starting Julia run the following to install the required packages:
# julia> Pkg.init(); for p in ("CUDAdrv","IJulia","Knet","PyCall","JLD2"); Pkg.add(p); end

In [32]:
Pkg.checkout("Knet","ilkarman") # make sure we have the right Knet version
Pkg.build("Knet")
using Knet
True=true # so we can read the python params
include("common/params_lstm.py");

INFO: Checking out Knet ilkarman...
INFO: Pulling Knet latest ilkarman...
INFO: No packages to install, update or remove
INFO: Building Knet
INFO: Compiling CUDA kernels.


make: `libknet8.so' is up to date.


In [33]:
println("OS: ", Sys.KERNEL)
println("Julia: ", VERSION)
println("Knet: ", Pkg.installed("Knet"))
println("GPU: ", readstring(`nvidia-smi --query-gpu=name --format=csv,noheader`))

OS: Linux
Julia: 0.6.1
Knet: 0.8.5+
GPU: Tesla K80
Tesla K80



In [34]:
# define model
function initmodel()
    rnnSpec,rnnWeights = rnninit(EMBEDSIZE,NUMHIDDEN; rnnType=:gru)
    inputMatrix = KnetArray(xavier(Float32,EMBEDSIZE,MAXFEATURES))
    outputMatrix = KnetArray(xavier(Float32,2,NUMHIDDEN))
    return rnnSpec,(rnnWeights,inputMatrix,outputMatrix)
end;

In [35]:
# define loss and its gradient
function predict(weights, inputs, rnnSpec)
    rnnWeights, inputMatrix, outputMatrix = weights # (1,1,W), (X,V), (2,H)
    indices = hcat(inputs...)' # (B,T)
    rnnInput = inputMatrix[:,indices] # (X,B,T)
    rnnOutput = rnnforw(rnnSpec, rnnWeights, rnnInput)[1] # (H,B,T)
    return outputMatrix * rnnOutput[:,:,end] # (2,H) * (H,B) = (2,B)
end

loss(w,x,y,r)=nll(predict(w,x,r),y)
lossgradient = grad(loss);

In [36]:
# load data
include(Knet.dir("data","imdb.jl"))
@time (xtrn,ytrn,xtst,ytst,imdbdict)=imdb(maxlen=MAXLEN,maxval=MAXFEATURES)
for d in (xtrn,ytrn,xtst,ytst); println(summary(d)); end

  3.460234 seconds (11.93 M allocations: 569.737 MiB, 5.75% gc time)
25000-element Array{Array{Int32,1},1}
25000-element Array{Int8,1}
25000-element Array{Array{Int32,1},1}
25000-element Array{Int8,1}


In [37]:
# prepare for training
weights = nothing; knetgc(); # Reclaim memory from previous run
rnnSpec,weights = initmodel()
optim = optimizers(weights, Adam; lr=LR, beta1=BETA_1, beta2=BETA_2, eps=EPS);

In [38]:
# force precompile (optional)
(x,y) = first(minibatch(xtrn,ytrn,BATCHSIZE))
@time lossgradient(weights,x,y,rnnSpec);

  0.055066 seconds (5.89 k allocations: 393.635 KiB)


In [39]:
info("Training...")
@time for epoch in 1:EPOCHS
    @time for (x,y) in minibatch(xtrn,ytrn,BATCHSIZE;shuffle=true)
        grads = lossgradient(weights,x,y,rnnSpec)
        update!(weights, grads, optim)
    end
end

INFO: Training...


  9.511471 seconds (365.05 k allocations: 44.876 MiB, 3.43% gc time)
  9.173121 seconds (367.03 k allocations: 44.604 MiB, 5.90% gc time)
  9.171262 seconds (367.76 k allocations: 44.615 MiB, 5.87% gc time)
 27.857207 seconds (1.11 M allocations: 134.422 MiB, 5.05% gc time)


In [40]:
info("Testing...")
total = correct = 0
@time for (x,y) in minibatch(xtst,ytst,BATCHSIZE)
    total += accuracy(predict(weights,x,rnnSpec), y; average=false)
    correct += length(y)
end
total/correct

INFO: Testing...


  2.778597 seconds (68.83 k allocations: 34.881 MiB, 7.74% gc time)


0.8534455128205128